In [1]:
%%sql
drop table if exists test_inventory_records


StatementMeta(, 22befdd5-7466-411b-8755-323b0c7db6bf, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
%%sql 
INSERT INTO starting_inventory_records (
    TCLAVE,
    ICLAVE,
    ATMFECHA,
    movements_sum,
    movements_sum_costo,
    cantidad,
    end_of_day,
    costo_unitario,
    inventario_inicial_costo,
    inventario_final_costo,
    new_products
)
WITH min_date AS (
    SELECT 
        store_id_from AS sid,
        product_id AS p_id,
        MIN(movement_date) AS md
    FROM fmovarti_intermediate_silver s
    LEFT JOIN starting_inventory_records i 
        ON s.store_id_from = i.TCLAVE
        AND s.product_id = i.ICLAVE
    WHERE 
        movement_date >= '2024-01-01'
        AND i.ICLAVE IS NULL
    GROUP BY store_id_from, product_id, i.TCLAVE, i.ICLAVE
)
SELECT 
    fis.store_id_from AS TCLAVE,
    fis.product_id AS ICLAVE,
    fis.movement_date AS ATMFECHA,
    -1 * SUM(fis.qty) AS movements_sum,
    SUM(fis.cost_amt) AS movements_sum_costo,
    0 AS cantidad,
    -1 * SUM(fis.qty) AS end_of_day,
    COALESCE(-1 * (SUM(fis.cost_amt) / NULLIF(SUM(fis.qty), 0)), 0) AS costo_unitario,
    0 AS inventario_inicial_costo,
    SUM(fis.cost_amt) AS inventario_final_costo,
    1 AS new_products
FROM fmovarti_intermediate_silver fis
JOIN min_date 
    ON fis.store_id_from = min_date.sid 
    AND fis.product_id = min_date.p_id
    AND fis.movement_date = min_date.md
GROUP BY 
    fis.store_id_from, 
    fis.product_id, 
    fis.movement_date;


StatementMeta(, 22befdd5-7466-411b-8755-323b0c7db6bf, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>